In [1]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
import numpy as np
import timeit
from math import *

a = 3
s = 3
r = 3
rr = 20
ro = 70
ra = 300
noise = 0.01
prop = 0.4
weight = 3
bias = np.array([0.0,-1.0])
dev_bias = 3
dTheta = 120
sight_theta = 180


In [ ]:
def initialize_agents(speed, N, width, height):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    seed()

    agents = [np.array([0.0, 0.0]) for i in range(N)]
    #agents = [Point(uniform(0, width), uniform(0, height)) for i in range(N)]
    
    speeds = N*[0,0]
    #speeds = [np.array([0.0, 0.0]) for i in range(N)]
    
    for i in range(N):
        theta = uniform(0, 2 * np.pi)
        #speeds[i][0] = speed * np.cos(theta)
        #speeds[i][1] = speed * np.sin(theta)
        speeds[i]= speed * np.array([np.cos(theta),np.sin(theta)])
        
        theta = uniform(0, 2 * np.pi)
        radius = uniform(0,height/2)
        agents[i] = Point(width/2 + radius*np.cos(theta), height/2 + radius*np.sin(theta))

    return agents, speeds


def initialize_window(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
        weightedSpeed = a * speeds[nearest_neighbours[i]]
        speeds[i] = speeds[i] + weightedSpeed
        speeds[i] = s * normalized(speeds[i])


def angle(vec1, vec2):  # (Not so) stupid way of doing an angle
    return np.arccos(np.dot(normalized(vec1),normalized(vec2))) # radians
            
def get_distances(agent, agents, N):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    dists = N * [0.0]
    for i in range(N):
        a, dists[i] = relative_pos(agent, agents[i])
        if dists[i] == 0:
            dists[i] = 0.1
    
    return dists

def relative_pos(agent1, agent2):
    dx = agent2.getX() - agent1.getX()
    dy = agent2.getY() - agent1.getY()

    return np.array([dx, dy]), np.linalg.norm([dx, dy])

def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents, N)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def softened_angle(speed, newspeed, s, maxTheta):
    theta = angle(speed, newspeed)
    if maxTheta**2 > theta**2:
        return s*newspeed
    else:
        return np.dot(rot_matrix(maxTheta), speed)    

def in_sight_range(rel_pos, speed1, angle_range):
    return 360*angle(speed1, rel_pos)/np.pi < angle_range
    

def noisy_vector(noise):
    return noise * np.array([2 * random() - 1, 2 * random() - 1])
    
def biaser(speeds, N, s, i, prop, bias, dev_bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * prop)
    gbias = np.random.normal(bias, dev_bias, )
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * gbias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir
    #bias = np.dot(tot_dir,np.array([[np.cos(rot_bias*i), 0],[0, np.sin(rot_bias*i)]]))
    #return bias

def quality (agents, speeds, N, bias, window, old_cm):
#ACCURACY (DIRECTIONS DISTRIBUTION)
    dev = 0.0
    for i in range(N):
        dev += angle(bias, speeds[i])
    dev_avg = dev/(N*2*(np.pi))
    #print (dev_avg)
    
#ELONGATION: SHAPE OF SWARM
    #can be smarter if we make agents become poses before, more globally in  the code
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    #Center of Mass
    cm = np.mean(poses,axis=0)
    #Standard Deviation
    std = np.std(poses,axis=0)
    #print (cm, std)

    #Elongation >1 means 
    elong = std[1] / std[0]
    #Drawing it
    #old_cm = Point(cm[0],cm[1])
    #old_cm.draw(window)
    #old_cm.setFill("red")

#GROUP DIRECTION
    #Vector
    group_dir = np.array([cm[0] - old_cm[0], cm[1] - old_cm[1]])                
    #print (group_dir)
    # Norm
    group_dir = np.linalg.norm(group_dir)
    #print (group_dir)
    return cm
    


def rigid_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


def rot_matrix(theta, unit = "None"):
    if unit in ["None", "deg"]:
        c, s = np.cos(pi * theta / 180), np.sin(pi * theta / 180)
    elif unit == "rad":
        c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],[s, c]])

def warn_me_args(N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dTheta):
    if dt*s < rr:
        print("Warning - step length bigger the repulsion radius.")
        

        
def virtualizer (current, agents, h, w, N):
    lower_limit = current.getY() - h / 2
    upper_limit = current.getY() + h / 2
    left_limit = current.getX() - w / 2
    right_limit = current.getX() + w / 2
    
    vagents=[np.array([agent.getX(),agent.getY()]) for agent in agents]
    virtuals = N * [0.0, 0.0]
    vvirtuals = N * [Point(0.0, 0.0)]

    for j in range(N):
        #make more compact!!!
        virtuals[j]=vagents[j]
        
        #newcopy = agents[i].clone()
        candidates = [vagents[j],
                      vagents[j]+[w,0],vagents[j]+[-w,0],vagents[j]+[w,h],vagents[j]+[w,-h],
                      vagents[j]+[-w,h],vagents[j]+[-w,-h],vagents[j]+[0,h],vagents[j]+[0,-h]]
        #print candidates
        
        #virtuals[j] = next((cand for cand in candidates if lower_limit < cand[1] < upper_limit and left_limit < cand[0] < right_limit),False
        for i in range(9):
            if lower_limit < candidates[i][1] < upper_limit and left_limit < candidates[i][0] < right_limit:
                virtuals[j]=candidates[i]
            
        vvirtuals[j] = Point(virtuals[j][0],virtuals[j][1])
        
    return vvirtuals


def get_cm_std(agents, N):
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    return np.mean(poses,axis = 0), np.std(poses,axis = 0)

def mill_observables (N, agents, speeds):
    cm, std = get_cm_std(agents, N)
    mean_R = np.linalg.norm(std)
    point_cm = Point(cm[0],cm[1])
    norm_R = N*[0.0]
    vector_R = N*[0,0]
    versor_R = N*[0,0]
    versor_T = N*[0,0]
    proj_radial = N*[0]
    radial = N*[0,0]
    proj_tangencial= N*[0]
#    norm_tangencial= N*[0]
    tangencial= N*[0,0]
    angles = N*[0.0]
    mean_angle = 0.0
    mean_speed = 0.0
    mean_tangencial = 0.0
    mean_radial = 0.0
    mean_square = 0.0
    angular= N*[0.0]
    
    for i in range(N):
        vector_R[i],norm_R[i] = relative_pos(point_cm, agents[i])
        
        versor_R[i] = normalized(vector_R[i])
        proj_radial[i] = np.dot(speeds[i], versor_R[i])
        radial[i] = proj_radial[i]*versor_R[i]
        
        angular[i] = (1/(norm_R[i])**2)*np.cross([vector_R[i][0],vector_R[i][1],0],[speeds[i][0],speeds[i][1],0])[2]
        #norm_tangencial[i]= norm_R[i]*angular[i]
        
        versor_T[i] = np.dot(versor_R[i],rot_matrix(90,'deg'))
        proj_tangencial[i] = np.dot(speeds[i], versor_T[i])
        tangencial[i] = proj_tangencial[i]*versor_T[i]
        
        
        angles[i] = (np.pi - angle(vector_R[i],speeds[i]))*360/(2*np.pi)
        mean_angle += angles[i]
        mean_tangencial += abs(proj_tangencial[i])
        mean_radial += abs(proj_radial[i])
        mean_speed += np.linalg.norm(speeds[i])
        mean_square += (np.linalg.norm(speeds[i]))**2
    
        #print proj_radial[i], proj_tangencial[i]
        #print angles[i], angular[i]
    mean_angle = mean_angle / N
    mean_speed = mean_speed / N
    mean_radial = mean_radial / N
    mean_tangencial = mean_tangencial / N
    mean_square = mean_square / N
    
    min_R = min(norm_R)
    max_R = max(norm_R)
    
    #return mean_R, min_R, max_R, 
    #return angles



In [53]:

##MILL MODEL
def mill (dt, agents, speeds, N, width, height, cr, ca, lr, la, alpha, beta, mass):
    clr = cr / lr
    cla = ca / la
    for i in range(N):
        u_dir = np.array([0.0, 0.0])
        propulsion = np.array([0.0, 0.0])
        friction = np.array([0.0, 0.0])
        grad_U = np.array([0.0, 0.0])
        
        #virtuals = virtualizer(agents[i], agents, height, width, N)
        
        for j in range(N):   #Duality interactions, by the Morse potential  
            if i == j:
                #Eliminate the i-i interaction
                continue

            rel_pos, distance = relative_pos(agents[i], agents[j])
            u_dir = normalized(rel_pos)
            grad_U = grad_U + u_dir * (clr*np.exp(-distance / lr) - cla *np.exp(- distance / la))
            
        
        propulsion = alpha * speeds[i] # self-propulsion propto alpha
        norm = (np.linalg.norm(speeds[i]))
        friction =  beta *((norm)**2) * speeds[i] #friction force propto beta
     
        d_speed = (propulsion - friction - grad_U)/mass
        speeds[i]= speeds[i]+dt*d_speed
        #print u_dir
    
   # print speeds[N-1], propulsion, friction, grad_U
    return

def run(N_steps, N, s, dt, width, height):#N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dev_bias, dTheta, sight_theta):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """
    
    agents, speeds = initialize_agents(s, N, width, height)
    window = initialize_window(agents, width, height)
    old_cm=np.array([0.0,0.0])
    bias = np.array([0.0,-1.0])
    for i in range(N_steps):
        
        #print 'step', i
        
        next_step(agents, speeds, dt, N)
        start = timeit.default_timer()
        ## MODEL
        #couple_speeds(agents, speeds, a, s, N)
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        #couzin(agents, speeds, N, width, height, s, noise, dTheta, rr, ro, ra, sight_theta, 1, 200, 1.5, 1) #(...)last 4 parameters: model2, roa, orient, atract
        
        #mill (dt, agents, speeds, N, width, height, cr=100, ca=150, lr=80, la=200.0, alpha=1.0, beta=0.01, mass=1) 
        # Region I, 
        #mill (dt, agents, speeds, N, width, height, cr=120, ca=250, lr=120, la=250.0, alpha=1.0, beta=0.01, mass=1) 
        # Region II, really nice ring
        mill (dt, agents, speeds, N, width, height, cr=120, ca=250, lr=230, la=150.0, alpha=2.0, beta=0.01, mass=3) 
        # Region IV, really nice ring
        #mill (dt, agents, speeds, N, width, height, cr=230, ca=150, lr=230, la=150.0, alpha=0.1, beta=0.01, mass=1) 
        # Region V, spreading
        #mill (dt, agents, speeds, N, width, height, cr=250, ca=100, lr=150, la=200.0, alpha=0.1, beta=0.1, mass=1) 
        # Region VIb, spreading
        #mill (dt, agents, speeds, N, width, height, cr=110, ca=100, lr=20, la=100.0, alpha=4.0, beta=0.06, mass=3) 
        # Region VII,(l<1/sqrt(C)) alpha ~ beta, best vortex in this region, decreasing in size with increasing N
        
        #mill_observables (N, agents, speeds)
        
        ## BOUNDARY CONDITIONS
        rigid_boundary(width, height, agents, speeds, N)
        #periodic_boundary(width, height, agents, speeds, N)

        #time.sleep(0.02)
        stop = timeit.default_timer()
        #print stop - start
    window.close()
    return

In [ ]:

run(2000, 30, 5, 0.1, 1000, 700)

KeyboardInterrupt: 